In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import datetime as dt

import music_generator.analysis.preprocessing as pp
from music_generator.analysis.play import play_array

import music_generator.analysis.stft as stft

In [ ]:
# https://blog.keras.io/building-autoencoders-in-keras.html

import keras

from keras.layers import Dense, Dropout, PReLU, Input
from keras.models import Model
from keras.optimizers import Adam

from keras.regularizers import l2
from keras.models import load_model

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras.layers import Input, Dense, Lambda
from keras.models import Model, Sequential
from keras import backend as K
from keras import metrics
from keras.datasets import mnist, fashion_mnist
from sklearn.preprocessing import MinMaxScaler

from keras.layers import LSTM, Lambda

In [ ]:
window = 1024
lstm_dim = 64
latent_dim = 2
batch_size = 32

In [ ]:
# sampling_rate, data = pp.read_wave_file("../data/Verplichte Kots_v9.wav", 0)
sampling_rate, data = pp.read_wave_file("../data/Dream Theater - Another day-fm7ntyycGbU.wav", 0)
# play_array(data, sampling_rate, range_secs=[17, 31])
# data = data[15*44100:29*44100]

In [ ]:
x_train = pp.create_batches(data, window, window)
x_train = np.array(x_train)
x_train = x_train.reshape(-1, window, 1)
x_train = x_train[0:(len(x_train) // batch_size) * batch_size]

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    noise = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
    return z_mean + K.exp(z_log_var / 2) * noise

In [ ]:
from keras.models import load_model

In [ ]:
# time-frequency model
tf_model = load_model('freq_loss')

In [ ]:
tf_model.layers.pop()
tf_model.trainable = False
tf_model.compile('Adam', 'mse')

In [ ]:
def custom_loss(y_true, y_pred):
#     xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
#     mse_loss = window * metrics.mse(y_true, y_pred)
#     kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
#     vae_loss = K.mean(mse_loss + 0.001 * kl_loss)
#     print(f"{y_true.shape}")
    
    inp_out = K.concatenate([y_true, y_pred], axis=-1)
    
    if str(tf_model.input.shape[0]) == '?':
        return K.mean(K.square(tf_model.predict(K.eval(inp_out))[0]))
    else:
        return K.mean(K.square(tf_model.predict(K.eval(inp_out))[0]))
    
#     return 

In [ ]:
tf_model.input.op.values #  K.zeros(shape=[32, 1024, 2])
# tf_model.output

In [ ]:
inp = Input(batch_shape=(batch_size, window, 1))
lstm = LSTM(lstm_dim, 
            stateful=True, 
            return_sequences=False,
            batch_input_shape=(batch_size, window, 1))(inp)

inter = Dense(lstm_dim, activation='relu')(lstm)
# inter = lstm

z_mean = Dense(latent_dim)(inter)
z_log_var = Dense(latent_dim)(inter)
z_sampled = Lambda(sampling, (latent_dim,))([z_mean, z_log_var])

decoder = Dense(512, activation='relu')
decoder_mean = Dense(window, activation='linear')

decoded = decoder(z_sampled)
# decoded = z_sampled




model = Model(inp, loss)

model.summary()
# model.compile('adam', loss='mse')


In [ ]:
ii = 0
while True:
    model.fit(x_train, x_train.reshape(-1, window), epochs=30, batch_size=batch_size, verbose=1)
    print("Storing to file...")
    y_pred = model.predict(x_train)
    # result = pp.batches_to_array(y_pred, window)
    result = y_pred.reshape(-1)
    pp.store_wave_file(f'temp_{ii}.wav', 44100, result)    
    ii += 1

# Deep & heavy model

In [ ]:
encoder = Model(inp, z_mean)


In [ ]:
x_encoded = encoder.predict(x_train)

In [ ]:
x_encoded_backup = x_encoded.copy()

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
x_encoded = x_encoded_backup[2000:3000]

trace = [go.Scatter3d(x=x_encoded[:, 0], 
                      y=x_encoded[:, 1], 
                      z=np.arange(len(x_encoded)),
                      mode='line',
                        marker=dict(
                            size=3,
                            color=np.arange(len(x_encoded)),                # set color to an array/list of desired values
                            colorscale='Viridis',   # choose a colorscale
                            opacity=0.8
                        ))]

fig = go.Figure(data=trace, layout=go.Layout(height=800, width=1000))
iplot(fig)

In [ ]:
# inp = Input(shape=(window, 1))
# lstm = LSTM(128, return_sequences=False)(inp)
# inter = Dense(64, activation='relu')(lstm)
# inter = Dense(16, activation='relu')(inter)
# inter = Dense(2, activation='relu')(inter)
# inter = Dense(16, activation='relu')(inter)
# inter = Dense(64, activation='relu')(inter)
# out = Dense(window, activation='linear')(inter)

# model = Model(inp, out)
# model.compile('adam', 'mse')
# model.summary()